In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

import sys
from config.variables import user

In [2]:
training_code = 'EMG'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['tipo'] = pd.to_numeric(training['RegistrationNumber'], errors='coerce').notnull()
training = training[training['tipo']]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao', 'tipo']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
1654,00000021,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2020-12-05 09:51:53
1539,00000022,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-09-08 15:44:08
1464,00000024,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-08-04 15:28:16
1224,00000026,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2020-12-18 00:31:26
1195,00000027,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2020-08-06 11:35:56
...,...,...,...,...,...
1023,00044789,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2020-12-14 15:37:03
1667,00044791,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-12-05 14:41:36
1151,00044794,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2020-12-06 17:51:30
1427,00044795,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2021-01-08 14:42:39


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)
rosters

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...
importing: 2021-02...


,cif,code,act_type,airline,fleet,tail,leg_type,fr,to,dep,arr,pairingId
0,00010103,SOBREAVIS,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-01 06:05:00,6173246.0
1,00006285,FP,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-02 06:00:00,NaN
2,00022301,SOBREAVIS,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-01 12:00:00,6099354.0
3,00036105,FANI,A,NaN,NaN,NaN,NaN,BSB,BSB,2020-01-01 06:00:00,2020-01-02 06:00:00,NaN
4,00013384,SOBREAVIS,A,NaN,NaN,NaN,NaN,POA,POA,2020-01-01 06:00:00,2020-01-01 14:15:00,6173917.0
...,...,...,...,...,...,...,...,...,...,...,...,...
168952,00033725,G3 1921,L,G3,7M8,NaN,J,MAO,FOR,2021-03-01 05:50:00,2021-03-01 09:10:00,6782573.0
168953,00042226,G3 1921,L,G3,7M8,NaN,J,MAO,FOR,2021-03-01 05:50:00,2021-03-01 09:10:00,6782573.0
168954,00036093,G3 1921,L,G3,7M8,NaN,J,MAO,FOR,2021-03-01 05:50:00,2021-03-01 09:10:00,6782573.0
168955,00002560,G3 1921,L,G3,7M8,NaN,J,MAO,FOR,2021-03-01 05:50:00,2021-03-01 09:10:00,6782573.0


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
59900,00002720,INSS
59901,00002976,INSS
59907,00000402,INSS
59910,00001128,INSS
59911,00001211,INSS
...,...,...
61377,00044050,INSS
61400,00043260,INSS
61406,00034570,INSS
61417,00024130,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
59906,00000127,LIC-SREM
59930,00012897,LIC-SREM
59937,00011361,LIC-SREM
59940,00005833,LIC-SREM
59950,00008112,LIC-SREM
...,...,...
61374,00046293,LIC-SREM
61375,00046335,LIC-SREM
61391,00044939,LIC-SREM
61392,00044967,LIC-SREM


In [9]:
rosters_o = rosters[rosters['code'].isin(funcs.training_filter('EMG'))]
rosters_o = rosters_o.sort_values(by=['cif','dep'])
rosters_o = rosters_o[['cif', 'code', 'dep']]
rosters_o.columns = ['cif', 'code_on', 'date_on']
rosters_o = rosters_o.drop_duplicates(subset = ['cif'], keep = 'last')
rosters_o

,cif,code_on,date_on
67752,00000021,C-EMG-ON,2020-12-11 19:00:00
12701,00000022,C-EMG-ON,2020-08-03 16:30:00
18435,00000024,C-EMG-ON,2020-08-04 16:30:00
106422,00000026,C-EMG-ON,2020-12-17 20:20:00
34932,00000027,C-EMG-ON,2020-08-07 16:30:00
...,...,...,...
105469,00044789,C-EMG-ON,2020-12-17 16:30:00
67998,00044791,C-EMG-ON,2020-12-11 20:20:00
67997,00044794,C-EMG-ON,2020-12-11 20:20:00
109940,00044795,C-EMG-ON,2021-01-17 16:30:00


In [10]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
72,00010655,EMG,2019-12-31
99,00009146,EMG,2023-01-31
375,00046230,EMG,2022-09-30
378,00023615,EMG,2021-03-30
381,00023617,EMG,2021-03-31
...,...,...,...
55314,00008599,EMG,2021-04-30
55319,00006293,EMG,2022-10-31
55326,00032174,EMG,2020-12-29
55327,00023879,EMG,2022-10-31


In [11]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters_o, on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code_on,date_on,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,EMG,2021-09-30,NaN,NaN,NaN,NaT,NaN,NaN,INSS,NaN
1,00000021,GRABLER,CMT,IBX,GRU,EMG,2022-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2020-12-05 09:51:53,C-EMG-ON,2020-12-11 19:00:00,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,EMG,2021-08-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-09-08 15:44:08,C-EMG-ON,2020-08-03 16:30:00,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,EMG,2021-08-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-08-04 15:28:16,C-EMG-ON,2020-08-04 16:30:00,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,EMG,2022-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2020-12-18 00:31:26,C-EMG-ON,2020-12-17 20:20:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [12]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

# Ajuste de data do treinamento online
validation['date_on'] = validation['date_on'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['date_on'] = pd.to_datetime(validation['date_on'], format='%Y-%m-%d')
validation['date_on'] = validation['date_on'].dt.floor('D')

# Data da conclusao ON LINE
validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation = validation[validation['date_on'] < (pd.to_datetime(dt.date(dt.now())))]

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code_on,date_on,inss,lnr
6,00000029,HERBERT,CMT,IBX,CGH,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"7,00",2021-01-05,C-EMG-ON,2021-01-14,NaN,NaN
62,00000507,ALENCAR,CMT,IBX,CGH,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"7,00",2021-01-14,C-EMG-ON,2021-01-21,NaN,NaN
74,00000632,ANDRE LUIZ,CMT,IBX,GRU,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2021-01-22,C-EMG-ON,2021-01-26,NaN,NaN
87,00000758,DANIEL HENRIQUE,CMT,NaN,CGH,EMG,2017-12-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"0,00",2020-07-10,C-EMG-ON,2020-07-14,NaN,NaN
90,00000765,GAGO JR,CMT,NaN,CGH,EMG,2017-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2021-02-01,C-EMG-ON,2021-02-04,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,00041957,GABRIEL COSTA,COP,NaN,GIG,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2021-01-02,C-EMG-ON,2021-01-15,NaN,NaN
1655,00042932,RICARDO MENEZES,COP,NaN,CGH,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"7,00",2021-01-07,C-EMG-ON,2021-01-07,NaN,NaN
1671,00042958,LUIZ VICTOR,COP,NaN,GRU,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2021-01-22,C-EMG-ON,2021-01-22,NaN,NaN
1675,00042962,JOSE THIAGO,COP,NaN,CGH,EMG,2021-02-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2021-01-19,C-EMG-ON,2021-01-19,NaN,NaN


In [13]:
validation.to_csv(f'./data/output/{training_code}_ON.csv', index=False, sep=';', encoding='utf-8-sig')